In [1]:
from pathlib import Path
import orbax.checkpoint

ckpt_path = Path('/Users/juanp.lievanok./Desktop/CHAI/Cam/code/pobax/results/lightbulbs_test/lightbulbs_20_16_seed(2024)_time(20250710-023923)_f99f277e7e557570929fb2a7cb4ae55e')
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
restored = orbax_checkpointer.restore(ckpt_path) #TODO look for how this is used somewhere else in the codebase

train_state = restored['final_train_state']

In [2]:
import jax
import numpy as np

def print_train_state(train_state: dict, depth=0) -> None:
    if isinstance(train_state, dict):
        for item, value in train_state.items():
            if isinstance(value, np.ndarray):
                print("  " * depth + f"{item}: [...]")
            elif isinstance(value, dict):
                print("  " * depth + f"{item}:")
                print_train_state(value, depth=depth+1)
            elif isinstance(value, list):
                print("  " * depth + f"{item}: [")
                for element in value:
                    print_train_state(element, depth=depth+1)
                print("  " * depth + "]")
            elif isinstance(value, tuple):
                print("  " * depth + f"{item}: (")
                for element in value:
                    print_train_state(element, depth=depth+1)
                print("  " * depth + ")")
            else:
                print("  " * depth + f"{item}: {value}")
    elif isinstance(train_state, list):
        print("  " * depth + "[")
        for element in train_state:
            print_train_state(element, depth=depth+1)
        print("  " * depth + "]")
    elif isinstance(train_state, tuple):
        print("  " * depth + "(")
        for element in train_state:
            print_train_state(element, depth=depth+1)
        print("  " * depth + ")")
    else:
        print("  " * depth + f"{train_state}")

# type(restored['final_train_state']['opt_state'][1][0]['count'])
print_train_state(restored)


args:
  action_concat: True
  alpha: [...]
  anneal_lr: True
  clip_eps: 0.2
  debug: True
  default_max_steps_in_episode: 1000
  double_critic: True
  entropy_coeff: 0.01
  env: lightbulbs_20_16
  gamma: 0.99
  hidden_size: 256
  image_size: 32
  lambda0: [...]
  lambda1: [...]
  ld_weight: [...]
  lr: [...]
  max_grad_norm: 0.5
  memoryless: False
  minibatch_size: 128
  n_seeds: 5
  num_envs: 4
  num_epochs: 50
  num_eval_envs: 10
  num_minibatches: 4
  num_steps: 128
  perfect_memory: False
  platform: cpu
  save_checkpoints: False
  save_runner_state: False
  seed: 2024
  show_discounted: False
  steps_log_freq: 1
  study_name: lightbulbs_test
  total_steps: 4000000
  update_epochs: 4
  update_log_freq: 1
  vf_coeff: [...]
argument_order: [
  vf_coeff
  ld_weight
  alpha
  lambda1
  lambda0
  lr
  rng
]
final_train_state:
  opt_state: [
    None
    [
      count: [...]
      mu:
        params:
          Dense_0:
            bias: [...]
            kernel: [...]
          Discret

In [3]:
from pobax.models import get_gymnax_network_fn
from pobax.envs.jax.lightbulbs import LightBulbs

env = LightBulbs('/Users/juanp.lievanok./Desktop/CHAI/Cam/code/pobax/pobax/envs/configs/lightbulbs_20_16_config.json')
env_params = env.default_params

network_class, action_size = get_gymnax_network_fn(env, env_params)

print(network_class, action_size)

<class 'pobax.models.discrete.DiscreteActorCriticRNN'> 21


In [4]:
from pobax.models.discrete import DiscreteActorCriticRNN # I think it is this one because this is the one get_gymnax_network_fn returns

# load stuff from `restored` into a DiscreteActorCriticRNN 

args = restored['args']

model = DiscreteActorCriticRNN(
   action_dim    = action_size, # The value of action_size was given by the get_gymnax_network_fn 
   double_critic = args['double_critic'],
   hidden_size   = args['hidden_size'],
)

In [5]:
# Create an observation 
# This test is for lightbulbs_20_16 and therefore the observation space is a gymnax.environments.spaces.Box(0, 1, (self.size,)) with size = 20

import pobax.envs.jax.lightbulbs as lbs

test_env = lbs.LightBulbs("/Users/juanp.lievanok./Desktop/CHAI/Cam/code/pobax/pobax/envs/configs/lightbulbs_20_16_config.json")

key = jax.random.key(0)
obs, state = test_env.reset_env(key = key, params = None)
obs, state

import jax.numpy as jnp
noop = jnp.zeros(21).astype(dtype='int32')
noop.at[20].set(1)


obs, state

(Array([1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32),
 LightBulbsState(bulbs=Array([1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32), goal=Array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0], dtype=int32), t=Array(0, dtype=int32, weak_type=True)))

In [ ]:
# model.apply inputs should be: (params, hidden_state, (obs, done))
from pobax.models import ScannedRNN

h0 = ScannedRNN.initialize_carry(batch_size=1, hidden_size=args['hidden_size']) #This is a float32 array with every value equal to 0. 
# TODO What exactly is the scanned RNN? 

ts_dict = jax.tree.map(lambda x: x[0, 0, 0, 0, 0, 0, 0], restored['final_train_state'])
params = ts_dict['params'] # These two lines inspired by visualize_navix_policy.py 
# TODO ask david what is going on with that lambda function. 
# (It seems to be removing the first 7 values from everything in the restored tree). 

# Concatenate observation and previous action because that concatenation is the actual observation.
# (that is why it was expecting an input of size 41 = 20+21 = obs_dim + action_dim and not of size 20 = obs_dim)
obs_act = jnp.concatenate([obs, noop], axis = -1)
obs_act_batch = obs_act[None, None, :] #TODO: This changes the shape from (41,) to (1,1,41) but I'm not sure why this is necessary. 

# The `done` in the input signature of model.apply apparently can't be a straight boolean but has to be an array. 
done_batch = jnp.zeros((1, 1), dtype=bool) #TODO Also I don't quite get why this should be the shape of the `done` input. 
logits, pi, value = model.apply(params, h0, (obs_act_batch, done_batch)) # TODO this lets me feed a a single observation+previous_action and memory state. 
# I am able to get an action out of it but how do I extract a new memory state?
# How to feed a trajectory? (A loop where I manually change the states and input the new observations and memory states to the model? Will it "jit" and be quick?)

likely_action = pi.probs.argmax()
likely_action

Array(8, dtype=int32)

In [130]:
h0[0]

Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.